In [1]:
# Setting up environment
# ----------------------
# Import libraries
import psycopg2
import cloudscraper
import time
from bs4 import BeautifulSoup
import sys
import os

# Set custom path for secrets
sys.path.insert(0,"/home/nuclear/Github/PythonPrograms/Secrets")

In [2]:
# Feedback for sql commands
# -------------------------
# Function to run try-except wrapped sql commands 
def run_sql (conn, sql_cmnd):
    cur = conn.cursor()
    try:
        cur.execute(sql_cmnd)
        print('success: ' + sql_cmnd)
    except:
        print('FAIL: ' + sql_cmnd)
    conn.commit()
    return 0
    

In [3]:
# Connecting to local database
# ----------------------------
# Connect to database
import crawler_pg_secrets as p
pg_params = {
    'host' : p.host,
    'user' : p.user,
    'dbname' : p.dbname,
    'password' : p.password,
    'port' : p.port
}
conn =  psycopg2.connect(**pg_params)
cur = conn.cursor()
conn.commit()

In [4]:
# Creating job url table
# ----------------------
table_name = 'indeed_horizon'

# Drop previous horizon table
sql_cmnd = "drop table " + table_name + ";"
run_sql(conn,sql_cmnd)

# Create new horizon table
sql_cmnd = "create table "\
    + table_name\
    + " (id serial, url varchar(2046), visited integer);"
run_sql(conn,sql_cmnd)

# Create MD5 hashed index
sql_cmnd = 'CREATE UNIQUE INDEX url_md5 ON indeed_horizon(MD5(url));'
run_sql(conn,sql_cmnd)

success: drop table indeed_horizon;
success: create table indeed_horizon (id serial, url varchar(2046), visited integer);
success: CREATE UNIQUE INDEX url_md5 ON indeed_horizon(MD5(url));


0

In [5]:
# Generate page urls
# -------------------
# Defining indeed endpoint, example: jobs?q=data+analyst&l=Toronto'&start=40
def create_end_url(job = 'data analyst', location = 'toronto', page = 3):
    connector_01 = '&l='
    connector_02 = "&start="
    job_url = job.strip().replace(" ","+")
    page_url = (page - 1) *10
    location_url = location
    end_url = '/jobs?q=' + job_url + connector_01 + location_url + connector_02 + str(page_url)
    print(end_url)
    return end_url

# Defining indeed full url, example: https://ca.indeed.com/jobs?q=data+analyst&l=Toronto'&start=40
def create_full_url(end_url):
    base_url = 'https://ca.indeed.com'
    full_url =  base_url + end_url
    print(full_url)
    return full_url

end_url = create_end_url('data entry', 'toronto',1) # testing url = create_url('data entry', 'ottawa',9)
full_url = create_full_url(end_url)


/jobs?q=data+entry&l=toronto&start=0
https://ca.indeed.com/jobs?q=data+entry&l=toronto&start=0


In [6]:
#Build table of horizons
# -----------------------
# Function to enter url into database
def db_input (url, visited = 0, table_name = 'indeed_horizon'):
    sql_cmnd = "insert into indeed_horizon (url, visited) values ('" + url + "'," +str(visited)+ ");"
    exit_status = run_sql(conn,sql_cmnd)
    return exit_status

# Extract urls from indeed
# ------------------------
# Parse page (from response object) for job urls
# def scrape_response(response):
def scrape_soup(soup): #, advert_count):
    job_url_base = 'https://ca.indeed.com'
    # soup = BeautifulSoup(response.text,"html.parser")
    ident='jcs-JobTitle css-jspxzf eu4oa1w0'
    class_link = soup.find_all(class_=ident) # Div class containing job link

    for count,link in enumerate(class_link):
        job_url_end = link.get('href')
        job_url = job_url_base + job_url_end
        exit_status = db_input(job_url)
        #print("advert_count before, scrape_soup: ",  advert_count)
        # if exit_status == 0:
        #     advert_count += 1
        #print("advert_count after, scrape_soup: ",  advert_count)
        print(count)
        print(job_url)
        #return advert_count

In [7]:
# Scraping with cloudscraper
# --------------------------
# Scraping instance
print(full_url)
scraper = cloudscraper.create_scraper()

# Cycling through pages
exit_flag = 0
advert_count = 0
max_adverts = 500
while exit_flag == 0:  #or advert_count < max_adverts:
    response = scraper.get(full_url)
    soup = BeautifulSoup(response.text,"html.parser")
    # Extract adverts from page
    #print("advert_count before, in page loop: ",  advert_count)
    scrape_soup(soup) #advert_count = scrape_soup(soup, advert_count)
    #print("advert_count after, in page loop: ",  advert_count)
    print(response.status_code)
    # Find next page
    try:
        next_page_resultset = soup.find_all(attrs = {'data-testid':'pagination-page-next'})
        next_page_end_link = next_page_resultset[0]['href']
        print("-"*20)
        print("next page end link ", next_page_end_link)
        print("-"*20)
        time.sleep(1)
        full_url = create_full_url(next_page_end_link)
    except:
        exit_flag = 1


https://ca.indeed.com/jobs?q=data+entry&l=toronto&start=0
advert_count before, in page loop:  0
success: insert into indeed_horizon (url, visited) values ('https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dm9JponA1i0YWpRNDcnDGwwM5SrUWUyuAiGnbl36WgLyAICksxnXW7fVTTHO96ChGqXugVlMsXuq2jPR1782gUjfw6og384qUQ5zKyzMHlI5Uq9i4tSdsxwmhm1L4NdGW5D2SbEjSAiplnSvDLY9P2_k2AjaWjUNYgjntGDxyKhYSJ53z_2u_RnZmf_-EsRgt9TUDbXMC2S23JW72-lLZdm7j3uznMM5I7LifLMsu8cIGgzU_l28FNZiPM_oP8jgvPVLf1zEGnlo07UE1E_4t81xImBeC8X3tXpIo9bWmdAj3EFW7Wb90yjqWBqiEsiFLzLiWFlmI7nXaCFe7rwO6LaRy3PaK2rTCSSFkho6e_GA6QkjfrMxak1ijDPHzBNvFh-INZMi49rxnddtv7pGEzvqvNsqbNtAYQjERwtWAUERix9jkSMHBy4l9SgHyyRbvKWVDB9f7fQcT5MOGHAYi3hZfvVvgzGkt-acJXBtMI97T_wLgrIc52QHtSiDz0p-4=&xkcb=SoCV-_M3X8-03T33W50LbzkdCdPP&p=0&fvj=1&vjs=3',0);
advert_count before, scrape_soup:  0
advert_count after, scrape_soup:  1
0
https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dm9JponA1i0YWpRNDcnDGwwM5SrUWUyuAiGnbl36WgLyAICksxnXW7fVTTHO96ChGqXugVlMsXuq2jPR1782gUjfw6og384qU

TypeError: '<' not supported between instances of 'NoneType' and 'int'


## Testing